Import TensorFlow 2

In [20]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import tensorflow as tf
print("TensorFlow version:", tf.__version__)

from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model

TensorFlow version: 2.11.0


Load and prepare the [MNIST dataset](http://yann.lecun.com/exdb/mnist/).

In [21]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# Add a channels dimension
x_train = x_train[..., tf.newaxis].astype("float32")
x_test = x_test[..., tf.newaxis].astype("float32")

Use `tf.data` to batch and shuffle the dataset:

Note: By using `.shuffle` we make the iteration over the dataset (batches, labels) to both be randomely sampled each time. Moreover, every epoch again the iteration is random.

In [22]:
batch_size = 64

train_ds = tf.data.Dataset \
    .from_tensor_slices((x_train, y_train)) \
    .shuffle(buffer_size=10*batch_size).batch(batch_size)

test_ds = tf.data.Dataset \
    .from_tensor_slices((x_test, y_test)) \
    .batch(batch_size)

Build the `tf.keras` model using the Keras [model subclassing API](https://www.tensorflow.org/guide/keras/custom_layers_and_models):

In [23]:
class CNN(Model):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = Conv2D(32, 3, activation='relu')
        self.flatten = Flatten()
        self.d1 = Dense(128, activation='relu')
        self.d2 = Dense(10)

    def call(self, x):
        x = self.conv1(x)
        x = self.flatten(x)
        x = self.d1(x)
        return self.d2(x)

# Create an instance of the model
model = CNN()

Choose an optimizer and loss function for training: 

In [24]:
loss_object = tf.keras.losses \
    .SparseCategoricalCrossentropy(from_logits=True)

optimizer = tf.keras.optimizers.Adam()

Select metrics to measure the loss and the accuracy of the model. These metrics accumulate the values over epochs and then print the overall result.

In [25]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics \
    .SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics \
    .SparseCategoricalAccuracy(name='test_accuracy')

Use `tf.GradientTape` to train the model:

In [26]:
@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        # training=True is only needed if there are layers with different
        # behavior during training versus inference (e.g. Dropout).
        predictions = model(images, training=True)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    train_loss(loss)
    train_accuracy(labels, predictions)

Test the model:

In [27]:
@tf.function
def test_step(images, labels):
    # training=False is only needed if there are layers with different
    # behavior during training versus inference (e.g. Dropout).
    predictions = model(images, training=False)
    t_loss = loss_object(labels, predictions)

    test_loss(t_loss)
    test_accuracy(labels, predictions)

In [28]:
EPOCHS = 5

for epoch in range(EPOCHS):
    # Reset the metrics at the start of the next epoch
    train_loss.reset_states()
    train_accuracy.reset_states()

    for images, labels in train_ds:
        train_step(images, labels)

    for test_images, test_labels in test_ds:
        test_step(test_images, test_labels)

    print(
        f'Epoch {epoch + 1}, '
        f'Loss: {train_loss.result()}, '
        f'Accuracy: {train_accuracy.result() * 100}, '
        f'Test Loss: {test_loss.result()}, '
        f'Test Accuracy: {test_accuracy.result() * 100}'
    )

Epoch 1, Loss: 0.14928539097309113, Accuracy: 95.5616683959961, Test Loss: 0.07143795490264893, Test Accuracy: 97.63999938964844
Epoch 2, Loss: 0.04537223279476166, Accuracy: 98.59333038330078, Test Loss: 0.06341726332902908, Test Accuracy: 97.90999603271484
Epoch 3, Loss: 0.022863328456878662, Accuracy: 99.31832885742188, Test Loss: 0.06259457767009735, Test Accuracy: 97.98999786376953
Epoch 4, Loss: 0.013765102252364159, Accuracy: 99.57666015625, Test Loss: 0.061319928616285324, Test Accuracy: 98.0875015258789
Epoch 5, Loss: 0.009890632703900337, Accuracy: 99.68666076660156, Test Loss: 0.060696348547935486, Test Accuracy: 98.14599609375


In [29]:
model.summary()

Model: "cnn_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           multiple                  320       
                                                                 
 flatten_2 (Flatten)         multiple                  0         
                                                                 
 dense_4 (Dense)             multiple                  2769024   
                                                                 
 dense_5 (Dense)             multiple                  1290      
                                                                 
Total params: 2,770,634
Trainable params: 2,770,634
Non-trainable params: 0
_________________________________________________________________
